<a href="https://colab.research.google.com/github/afoteygh/Research/blob/main/Testing%20Visualization_Run2_Publication_snortrulegenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Run2 Details**
Here are the details for this run 
Layers=12
Model= 355M


# Installing Dependencies
Run the cell below to install model dependencies

In [ ]:
# having problems with tf 2.0+

%tensorflow_version 1.x
!git clone https://github.com/Akings/snort_rule_gen.git
!pip install -r /content/snort_rule_gen/requirements.txt
from datetime import datetime
from google.colab import files

fatal: destination path 'snort_rule_gen' already exists and is not an empty directory.


In [ ]:
import tarfile
import os
import json
import requests
import sys
import shutil
import re
from tqdm import tqdm, trange
import numpy as np
import tensorflow as tf
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.python.client import device_lib
import time
import csv
import argparse

*Import* our utilities

In [ ]:
from snort_rule_gen.utilities import model, sample, encoder, memory_saving_gradients
from snort_rule_gen.utilities.load_dataset import load_dataset, Sampler
from snort_rule_gen.utilities.accumulate import AccumulatingOptimizer

function to download files from the internet with nice progress bar 

Import model code here

In [ ]:
from snort_rule_gen import srg

To Monitor the Colab

In [ ]:
from urllib.request import urlopen


In [ ]:
exec(urlopen("http://colab-monitor.smankusors.com/track.py").read())
_colabMonitor = ColabMonitor().start()



Now live at : http://colab-monitor.smankusors.com/5f92ecc8cdfc7


In [ ]:
srg.download_vocab(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 602Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 114Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 754Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:12, 114Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 702Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 130Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 196Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colab VM, and to get the trained model out of Colab, is to route it through Google Drive *first*.

Running this cell will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code)

In [ ]:
srg.mount_gdrive()

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




```
# This is formatted as code
```

## Uploading Rules File to be Trained
In the Colab sidebar on the left of the screen, select the folder icon. From there select dirve then  My Drive then you can upload files there
 Then run the cells below to copy file into colab

In [ ]:
file_name = "emerging-trojanOriginal.txt" #  filename of your file

In [ ]:
srg.copy_file_from_gdrive(file_name)

## Training config

The next cell creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `finetuning`:


*  **`restore_from`**: Set to `fresh` to start training from the base model, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
 

In [ ]:
sess = srg.start_tf_sess()

srg.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=1000,
              restore_from='fresh',
              run_name='run2',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:06<00:00,  6.83s/it]


dataset has 1408741 tokens
Training...
[10 | 18.91] loss=0.94 avg=0.94
[20 | 27.73] loss=0.63 avg=0.79
[30 | 36.55] loss=0.58 avg=0.72
[40 | 45.39] loss=1.49 avg=0.91
[50 | 54.21] loss=1.15 avg=0.96
[60 | 63.03] loss=0.95 avg=0.96
[70 | 71.87] loss=0.69 avg=0.92
[80 | 80.72] loss=1.10 avg=0.94
[90 | 89.53] loss=0.52 avg=0.89
[100 | 98.35] loss=1.08 avg=0.91
[110 | 107.18] loss=0.89 avg=0.91
[120 | 116.00] loss=1.10 avg=0.93
[130 | 124.83] loss=0.39 avg=0.88
[140 | 133.66] loss=0.98 avg=0.89
[150 | 142.50] loss=0.37 avg=0.85
[160 | 151.33] loss=0.18 avg=0.81
[170 | 160.14] loss=0.71 avg=0.80
[180 | 168.98] loss=1.03 avg=0.82
[190 | 177.79] loss=0.60 avg=0.80
[200 | 186.61] loss=0.88 avg=0.81
======== SAMPLE 1 ========
atalay_history /history/11/07/to-the-citizen-of-noruegaa/; classtype:trojan-activity; sid:2026561; rev:3; metadata:affected_product Windows_XP_Vista_7_8_10_Server_32_64_Bit, attack_target Client_Endpoint, deployment Perimeter, signature_severity Major, created_at 2018_03_3

After the model is trained, you can copy the checkpoint folder to your own Google Drive if you want to use it later.

If you want to download it to your personal computer, I recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally for use.

if you already have a checkpoint go to train model from checkpoint

In [ ]:
srg.copy_checkpoint_to_gdrive(run_name='run2')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into Colab

the filename should match `run2` in the celll below

In [ ]:
srg.copy_checkpoint_from_gdrive(run_name='run2')

sess = srg.start_tf_sess()
srg.load_gpt2(sess, run_name='run2')

Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000


## Generate Rules From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate rules. `generate` generates a single rules from the loaded model.

This is a simple test for a small number of rules just to explain some helpful parmeters

In [ ]:
srg.generate(sess, run_name='run2')

A 2nd Generation Generic Request using the SMTP Server (sendemail.net)"; flow:established,to_server; content:"Subject|3a 20|sendemail.net"; http_header; fast_pattern:only; reference:url,www.cylance.com/shell-crew-variants-continue-to-fly-under-big-avs-radar; classtype:trojan-activity; sid:2022375; rev:1; metadata:created_at 2015_08_20, updated_at 2019_09_28;)<|endoftext|>

<|startoftext|>alert tcp $HOME_NET any -> $EXTERNAL_NET $HTTP_PORTS (msg:"ET TROJAN LDP-1.0 HTTP Request (sendemail.net)"; flow:established,to_server; content:"/sendemail.net/"; http_uri; fast_pattern:only; content:"|3a 20|"; http_header; content:!"Referer|3a|"; http_header; content:!"Accept-"; http_header; content:!"Connection"; http_header; content:!"Content-"; http_header; pcre:"/^\/sendemail\.net\/\?[a-f0-9]{28}&[a-f0-9]{28}&[a-f0-9]{28}&\s[^&]+$/U"; reference:url,www.cylance.com/shell-crew-variants-continue-to-fly-under-big-avs-radar; classtype:trojan-activity; sid:2022377; rev:1; metadata:created_at 2015_08_20,

If you're creating an API based on your model and need to pass the generated rules elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the rules to start with a given character sequence and generate rules from there (good if you want to know where the rule starts).

You can also generate multiple rules at a time by specifing `nsamples`. you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (set a maximum of 20 for `batch_size` to avoid running out of memory).

Other optional-but-helpful parameters

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the output (default 0.7, please keep it between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

See how they are used below

In [ ]:
srg.generate(sess,
              length=1023,
              top_p=0.9,
              temperature=1.0,
              prefix="alert tcp",
              nsamples=5,
              batch_size=5
              )

alert tcp $HOME_NET any -> $EXTERNAL_NET $HTTP_PORTS (msg:"ET TROJAN Bladab User-Agent and MAC Check"; flow:established,to_server; content:"User-Agent|3a 20|Spam-Squawk/MAC-Check|0d 0a|"; http_header; fast_pattern; content:"Cookie|3a 20|KeyLogger|20|"; http_header; pcre:"/User-Agent\x3a\s-adf-mb3-ing-0d|MAC-Check$/Hi"; reference:md5,7b22c10d1f0a84345ccd68c6bf01; classtype:trojan-activity; sid:2027806; rev:2; metadata:created_at 2019_09_20, updated_at 2019_09_20;)<|endoftext|>

<|startoftext|>alert tcp $HOME_NET any -> $EXTERNAL_NET $HTTP_PORTS (msg:"ET TROJAN Ransomware.Win32.ssp.pl (Powered By Nyacwihrdot C&C)"; flow:established,to_server; content:"User-Agent|3a 20|ssp"; nocase; http_header; fast_pattern:19,20; content:"Cookie|3a 20|ID="; nocase; http_header; pcre:"/ID=\w+$/U"; reference:url,www.threatexpert.com/report.aspx?md5=fd867b7df4969431542d1791; reference:url,www.threatexpert.com/report.aspx?md5=fb5dad19ba55a8cdf4f74ad9; classtype:trojan-activity; sid:2027807; rev:2; metadata:

# Bulk Generation

For bulk generation, you can generate a large amounts rules to a `.txt` file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'snort_genrule_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

srg.generate_to_file(sess,
                      prefix="alert tcp",
                      destination_path=gen_file,
                      length=500,
                      top_p=0.9,
                      temperature=1.0,
                      nsamples=2048,
                      batch_size=32
                      )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
# to download the file
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>